In [ ]:
import getpass
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from datetime import datetime
import glob, os
import pandas as pd
from selenium.webdriver.common.keys import Keys
from openpyxl import load_workbook
import datetime
import math
import random
from openpyxl import load_workbook


In [ ]:


cnpjs = ['cnpj1','cnpj2']

for i in range(len(cnpjs)):
    print(str(i) + " - "+cnpjs[i])
try:
    escolha_cnpj = input("Qual o CNPJ deseja buscar? ")
    cnpj = cnpjs[int(escolha_cnpj)].replace(".","").replace("/","").replace("-","")
    #print(cnpj)
except:
    print("Opção Inválida.")

#abrir site, aguardar captcha entrar e capturar os valores
driver_exe = './chromedriver.exe'
browser = webdriver.Chrome(driver_exe)
browser.get('https://www.dividaativa.pge.sp.gov.br/sc/pages/consultas/consultarDebito.jsf')

from selenium.webdriver.support.select import Select
select_element = browser.find_element(By.ID,'consultaDebitoForm:decLblTipoConsulta:opcoesPesquisa')
select_object = Select(select_element)
select_object.select_by_index(1)

sleep(1)
acao = browser.find_element_by_id("consultaDebitoForm:decTxtTipoConsulta:cnpj")
acao.clear()
acao.send_keys(cnpj)

input("Resolver o captcha manualmente e apertar enter para continuar...")
print('Aguardar leitura dos dados...')

#ipva

#rotina do regex ipva
import re


In [ ]:

def trata_ipva(texto_pag):

    texto_trat = texto_pag.replace("\n","#")
    #print(texto_trat)
    nr_reg_re = re.compile('#\d{10}#')
    nr_proc_re = re.compile('Número(?:[^#]*#){2}')
    situ_re = re.compile('Situ(?:[^#]*#){2}')
    principal_re = re.compile('#Princ(?:[^#]*#){1}')
    juros_prin = re.compile ('#Jur(?:[^#]*#){1}')
    multa_mora = re.compile('#Mul(?:[^#]*#){1}')
    juros_more = re.compile('#Juros de Mora da(?:[^#]*#){1}')
    dados_veiculo = re.compile('#\D{3}\d{4}\s\S*\s(.*)|#\D{3}\d{1}\D{1}\d{2}\s\S*\s(.*)')
    placa_re = re.compile('#\D{3}\d{4}|#\D{3}\d{1}\D{1}\d{2}')
    segundo_espaco = re.compile('\s\S*\s(.*)')



    #print(nr_reg_re.search(texto_trat).group(0).replace("#","").strip())
    #print(nr_proc_re.search(texto_trat).group(0).replace('Número do Processo (Unificado):','').replace("#","").strip())
    #print(situ_re.search(texto_trat).group(0).replace('Situação:','').replace("#","").strip())
    #print(principal_re.search(texto_trat).group(0).replace('Principal','').replace("#","").strip())
    #print(juros_prin.search(texto_trat).group(0).replace('Juros de Mora do Principal','').replace("#","").strip())
    #print(multa_mora.search(texto_trat).group(0).replace('Multa de Mora do Principal','').replace("#","").strip())
    #print(juros_more.search(texto_trat).group(0).replace('Juros de Mora da Multa de Mora','').replace("#","").strip())
    #print(placa_re.search(dados_veiculo.search(texto_trat).group(0)).group(0).replace("#","").strip())
    #print(segundo_espaco.search(dados_veiculo.search(texto_trat).group(0).replace(dados_veiculo.search(texto_trat).group(1),"")).group(0).strip())   
    #print(dados_veiculo.search(texto_trat).group(1).replace('##########Ouvidoria#Transparência#SIC',""))

    nr_reg = nr_reg_re.search(texto_trat).group(0).replace("#","").strip()
    nr_proc = nr_proc_re.search(texto_trat).group(0).replace('Número do Processo (Unificado):','').replace("#","").strip()
    situ = situ_re.search(texto_trat).group(0).replace('Situação:','').replace("#","").strip()
    principal = principal_re.search(texto_trat).group(0).replace('Principal','').replace("#","").strip()
    juros_pri = juros_prin.search(texto_trat).group(0).replace('Juros de Mora do Principal','').replace("#","").strip()
    multa = multa_mora.search(texto_trat).group(0).replace('Multa de Mora do Principal','').replace("#","").strip()
    juros_mora = juros_more.search(texto_trat).group(0).replace('Juros de Mora da Multa de Mora','').replace("#","").strip()
    placa = placa_re.search(dados_veiculo.search(texto_trat).group(0)).group(0).replace("#","").strip()
    renavam = segundo_espaco.search(dados_veiculo.search(texto_trat).group(0).replace(dados_veiculo.search(texto_trat).group(1),"")).group(0).strip()   
    demais_dados = dados_veiculo.search(texto_trat).group(1).replace('##########Ouvidoria#Transparência#SIC',"")
    
    return (nr_reg,nr_proc,situ,principal,juros_pri,multa,juros_mora,placa,renavam,demais_dados)


In [ ]:
    
#multa ipca

#multas

#taxa judiciaria

#Loop para capturar as dívidas existentes
id_tp_dividas = []
tp_dividas = []
id_qtd_dividas = []
qtd_dividas = []

tempo = []
ids = browser.find_elements_by_xpath('//*[@id]')
#print(ids)
for cod in ids:
    if cod.get_attribute('id').find('218') >=0 or cod.get_attribute('id').find('248') >=0:    
            id_qtd_dividas.append(cod.get_attribute('id'))
            qtd_dividas.append(cod.get_attribute('innerText'))
    if cod.get_attribute('id').find('consultaDebitoForm:dataTable:') >=0 or cod.get_attribute('id').find('consultaDebitoForm:dataTableMuitosDebitosAgrupad') >=0:
        if cod.get_attribute('id').find('lnkCons') >=0:
            id_tp_dividas.append(cod.get_attribute('id'))
            tp_dividas.append(cod.get_attribute('innerText'))

print('Detalhes:\n')

for i in range(len(tp_dividas)):

    print(str(i) + ' - ' + tp_dividas[i] + '- Qtd: '+ qtd_dividas[i] + ' Págs: '+ str(math.floor(int(qtd_dividas[i])/500)+1) +' - Tempo estim.: ' + str(datetime.timedelta(seconds=(6*int(qtd_dividas[i])))))

x = input("Digite o número do débito correspondente que deseja extrair: ")

if tp_dividas[int(x)] == 'IPVA':
    print("IPVA")
else:
    print("Processo ainda não mapeado.")

#def ipva(x)
x = 0 #remover
print("Aguardar carregamento dos débitos.")
browser.find_element_by_id(id_tp_dividas[int(x)]).click()



a = str(random.randrange(1, 999)) + str(random.randrange(1, 999))

col_names =  ['Número Registro','Número Processo','Situação','Principal','Juros do Principal','Multa','Juros Mora','Placa','Renavam','Demais dados']
my_df  = pd.DataFrame(columns = col_names)
my_df.to_excel('./Saida_IPVA_'+ a + '.xlsx',index = False,header= True)


path = './Saida_IPVA_'+ a + '.xlsx'
book = load_workbook(path)
writer = pd.ExcelWriter('./Saida_IPVA_'+ a + '.xlsx', engine='openpyxl')
writer.book = book
writer.sheets = {ws.title: ws for ws in book.worksheets}

import random

pag_atual = 1
#define tipo do id dos nr_reg
if escolha_cnpj == '0':
    id_nr_pre = 'consultaDebitoForm:dataTable2:'
    id_nr_pos = ':j_id281'
else:
    id_nr_pre = 'consultaDebitoForm:dataTableMuitosDebitos:'
    id_nr_pos = ':j_id322'

#capturar todos os numeros de registros numa lista e clicar no link que tem esse texto
#ids = browser.find_elements_by_xpath('//*[@id]')
df_compilada = pd.DataFrame()
#depois construir rotina para pular páginas
for i in range(int(qtd_dividas[x])):#int(qtd_dividas[x]):
    
    #proxima página
    if i % 500 and i > 498 == 0:
        browser.find_element_by_tag_name('html').send_keys(Keys.END)
        #browser.find_element_by_link_text('»').click()
        browser.find_element(By.XPATH, '//td[text()="»"]').click()
        browser.find_element_by_tag_name('html').send_keys(Keys.HOME)
        browser.find_element_by_tag_name('html').send_keys(Keys.PAGE_DOWN)
        pag_atual += 1
        sleep(5)
            
    
    id_nr = id_nr_pre +str(i)+ id_nr_pos
    print('Colhendo registro:' + str(i))
    browser.find_element_by_id(id_nr).click()  

    texto_pag =  browser.find_element_by_xpath("/html/body").text
    html = browser.find_element_by_tag_name('html')
    html.send_keys(Keys.PAGE_DOWN)
    #esse page down meio q revalida a pagina para que os links possam ser encontrados
    #se o click falhou, tentar clicar novamente
    
    valida_pag_re = re.compile('Dados N° de Registro/CDA')
    try:
        valida_pag = valida_pag_re.search(texto_pag).group(0)
    except:
        valida_pag = ""
        
        
    if valida_pag == "":
        print('Não realizou revalidação da página.')
        html = browser.find_element_by_tag_name('html')
        html.send_keys(Keys.PAGE_DOWN)
        sleep(1)
        browser.find_element_by_id(id_nr).click()
        texto_pag =  browser.find_element_by_xpath("/html/body").text
        sleep(0.5)
    
    #print(texto_pag)
   
    #captura dados
    
    nr_reg,nr_proc,situ,principal,juros_pri,multa,juros_mora,placa,renavam,demais_dados = trata_ipva(texto_pag)
    df = pd.DataFrame({'Número Registro' : nr_reg, 'Número Processo':nr_proc,'Situação':situ,'Principal':principal,'Juros do Principal':juros_pri,'Multa':multa,'Juros Mora':juros_mora,'Placa':placa,'Renavam':renavam,'Demais dados':demais_dados},index=[0])
    
    #df.to_csv('log.csv', mode='a', index=False, header=False)
    df.to_excel(writer, startrow=writer.sheets['Sheet1'].max_row, index = False,header= False)
    writer.save()
    #df_compilada = df_compilada.append(df)
    
    #volta
    sleep(1)
    browser.find_element_by_id("consultaDebitoForm:btnVoltarDetalheDebito").click()
    sleep(6)
    texto_pag = ""
print('Processo finalizado!')
    #exporta data frames a cada 10
    #proxima iteração